In [ ]:
import gradio as gr
import pandas as pd
import random

simple = pd.DataFrame(
    {
        "a": ["A", "B", "C", "D", "E", "F", "G", "H", "I"],
        "b": [28, 55, 43, 91, 81, 53, 19, 87, 52],
    }
)

fake_barley = pd.DataFrame(
    {
        "site": [
            random.choice(
                [
                    "University Farm",
                    "Waseca",
                    "Morris",
                    "Crookston",
                    "Grand Rapids",
                    "Duluth",
                ]
            )
            for _ in range(120)
        ],
        "yield": [random.randint(25, 75) for _ in range(120)],
        "variety": [
            random.choice(
                [
                    "Manchuria",
                    "Wisconsin No. 38",
                    "Glabron",
                    "No. 457",
                    "No. 462",
                    "No. 475",
                ]
            )
            for _ in range(120)
        ],
        "year": [
            random.choice(
                [
                    "1931",
                    "1932",
                ]
            )
            for _ in range(120)
        ],
    }
)


def bar_plot_fn(display):
    if display == "simple":
        return gr.BarPlot(
            simple,
            x="a",
            y="b",
            title="Simple Bar Plot with made up data",
            tooltip=["a", "b"],
            y_lim=[20, 100],
        )
    elif display == "stacked":
        return gr.BarPlot(
            fake_barley,
            x="variety",
            y="yield",
            color="site",
            title="Barley Yield Data",
            tooltip=["variety", "site"],
        )
    elif display == "grouped":
        return gr.BarPlot(
            fake_barley.astype({"year": str}),
            x="year",
            y="yield",
            color="year",
            group="site",
            title="Barley Yield by Year and Site",
            group_title="",
            tooltip=["yield", "site", "year"],
        )
    elif display == "simple-horizontal":
        return gr.BarPlot(
            simple,
            x="a",
            y="b",
            x_title="Variable A",
            y_title="Variable B",
            title="Simple Bar Plot with made up data",
            tooltip=["a", "b"],
            vertical=False,
            y_lim=[20, 100],
        )
    elif display == "stacked-horizontal":
        return gr.BarPlot(
            fake_barley,
            x="variety",
            y="yield",
            color="site",
            title="Barley Yield Data",
            vertical=False,
            tooltip=["variety", "site"],
        )
    elif display == "grouped-horizontal":
        return gr.BarPlot(
            fake_barley.astype({"year": str}),
            x="year",
            y="yield",
            color="year",
            group="site",
            title="Barley Yield by Year and Site",
            group_title="",
            tooltip=["yield", "site", "year"],
            vertical=False,
        )


with gr.Blocks() as bar_plot:
    with gr.Row():
        with gr.Column():
            display = gr.Dropdown(
                choices=[
                    "simple",
                    "stacked",
                    "grouped",
                    "simple-horizontal",
                    "stacked-horizontal",
                    "grouped-horizontal",
                ],
                value="simple",
                label="Type of Bar Plot",
            )
        with gr.Column():
            plot = gr.BarPlot()
    display.change(bar_plot_fn, inputs=display, outputs=plot)
    bar_plot.load(fn=bar_plot_fn, inputs=display, outputs=plot)

bar_plot.launch()

	

In [ ]:
def score_report_bar():
    path=r"C:\Users\Inception\Desktop\LLM-based-QA-chatbot-builder\UI\score_report"
    import os
    import math
    dat=[]
    for x in os.listdir(path):
        wh=[]
        flag=0
        for x2 in x:
            if x2>='a' and x2<='z':
                flag=1
                wh.append(x2)
            elif flag==1:
                wh.append(" ")
        wh=''.join(wh)
        wh=wh.replace("model ans","")
        wh=wh.replace("finetuned","")
        wh=wh.replace("  "," ")
        wh=wh.replace("xlsx","")
        df_temp=pd.read_excel(os.path.join(path,x))
        rating=sum(df_temp["rating"])/len(df_temp)
        dat.append({
            "Model Name":wh,
            "Average Rating":rating
        })
    temp=pd.DataFrame(dat)
    return temp

In [ ]:
import gradio as gr
import pandas as pd

def bar_plot_fn():
    temp=score_report_bar()
    return gr.BarPlot(
            temp,
            x="Model Name",
            y="Average Rating",
            x_title="Model name",
            y_title="Average Rating",
            title="Simple Bar Plot with made up data",
            tooltip=["Model Name", "Average Rating"],
            y_lim=[1, 5],
            width=200,
            height=1000
        )
with gr.Blocks() as bar_plot:
    with gr.Row():
        btn=gr.Button("test")
    with gr.Row():
        plot = gr.BarPlot()
    btn.click(bar_plot_fn, None, outputs=plot)

bar_plot.launch()	

In [ ]:
def parse_data(link,num=None):    
    from bs4 import BeautifulSoup
    import requests
    import re
    from docx import Document       
    from langchain_community.document_loaders import WebBaseLoader
    s=set()
    import time
    start_time = time.time()
    duration = 5
    def get_links(url):
        response = requests.get(url)
        data = response.text
        soup = BeautifulSoup(data, 'lxml')

        links = []
        for link in soup.find_all('a'):
            link_url = link.get('href')
            if link_url is not None and link_url.startswith('http'):
                s.add(link_url)
                links.append(link_url)
        
        return links
    # def write_to_file(links):
    #     with open('data.txt', 'a') as f:
    #         f.writelines(links)
    def get_all_links(url):
            for link in get_links(url):
                if (time.time() - start_time) >= duration:
                    return
                get_all_links(link)

    def data_ret2(link):
        loader = WebBaseLoader(f"{link}")
        data = loader.load()
        return data[0].page_content
    # link = 'https://kuet.ac.bd'
    s.add(link)
    get_all_links(link)
    li=list(s)
    all_data=[]
    if num==None:
        num=len(li)
    for idx,x in enumerate(li):
        if idx==num:
            break
        try:
            print("Link: ",x)
            all_data.append(data_ret2(x))
        except:
            print("pass")
            continue
    all_data2 = re.sub(r'\n+', '\n\n', "\n".join(all_data))
    document = Document()
    document.add_paragraph(all_data2)
    document.save('docx_file.docx')

In [ ]:
import os
os.getcwd()

In [ ]:
df_all=[]
for x in os.listdir("save_ques_ans"):
    path=os.path.join("save_ques_ans",x)
    df_all.append(pd.read_excel(path))
df=pd.concat(df_all,axis=0)

In [ ]:
df

In [ ]:

doc=[]
for x in s:
    print(x)
    doc.extend(data_ret2(x))
    

In [ ]:
doc